In [2]:
from datasets import load_dataset
import streamlit as st

dataset = load_dataset("vicgalle/alpaca-gpt4")


C:\Users\cabro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(dataset["train"][0])  # Exemple d'un élément

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for sta

In [4]:
# Extraire questions et réponses
qa_pairs = [
    {"question": item["instruction"], "answer": item["output"]}
    for item in dataset["train"]
]

# Afficher les 5 premières paires
print(qa_pairs[:5])

[{'question': 'Give three tips for staying healthy.', 'answer': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'}, {'question': 'What are the three primary colors?', 'answer': 'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be crea

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    doc = nlp(text)
    cleaned = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct
    ]
    return " ".join(cleaned)

# Nettoyer les questions et réponses
for pair in qa_pairs:
    pair["cleaned_question"] = clean_text(pair["question"])
    pair["cleaned_answer"] = clean_text(pair["answer"])

# Exemple de données nettoyées
print(qa_pairs[:5])

In [1]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Tokenisation des questions et réponses
corpus = [word_tokenize(pair["cleaned_question"]) for pair in qa_pairs]
corpus += [word_tokenize(pair["cleaned_answer"]) for pair in qa_pairs]

print("Exemple de corpus tokenisé :", corpus[:5])


NameError: name 'qa_pairs' is not defined

In [6]:
# Entraîner un modèle Word2Vec
word2vec_model = Word2Vec(
    sentences=corpus,  # Corpus de texte
    vector_size=300,   # Taille des vecteurs
    window=5,          # Taille de la fenêtre de contexte
    min_count=2,       # Mots ignorés s'ils apparaissent moins de 2 fois
    workers=4          # Nombre de threads pour l'entraînement
)

# Sauvegarder le modèle pour réutilisation
word2vec_model.save("word2vec_model.model")
print("Modèle Word2Vec entraîné et sauvegardé.")


Modèle Word2Vec entraîné et sauvegardé.


In [7]:
# Charger un modèle Word2Vec sauvegardé
word2vec_model = Word2Vec.load("word2vec_model.model")

# Vérifier les vecteurs d'un mot
print(word2vec_model.wv["health"])  # Exemple pour le mot 'health'

[ 0.34830174 -0.68942875  1.8957697  -1.337736    0.14192504 -0.15096317
 -0.35081038  0.48723295 -0.03171167  2.7057714   1.1881399  -2.50578
 -0.79031545  1.3050103  -0.7731478   1.8828013  -0.7334777  -0.37587053
  1.2324599   1.0084045   0.6169321  -0.4868601  -0.17397475  2.1679754
 -0.35112992  0.31512225 -1.6387095   1.7071099   1.1167533  -0.8303334
  1.226821    0.21648526 -0.13620634  2.0616713   0.90552354 -0.42578095
  1.231223   -0.8723191   1.3670287   1.2949588  -0.86143833 -1.7298138
  0.75559306 -1.7263805  -1.7478775   0.00778423 -1.604616   -0.96309644
 -0.5170565   1.1199994  -0.29306415 -1.9125861   0.74876225 -1.6985742
 -0.01057893  1.0446043   0.35433415 -0.5394359   0.7438492   0.09379223
 -0.03571579  0.8535809  -1.2410282   0.9270205   0.13962589 -0.4861284
  2.1037004   1.0007379   0.0195886  -1.3223832  -1.325878    0.5015279
  0.13420543 -0.29212752 -1.1957299   0.26703838 -1.6463346  -0.35108617
  0.89651865 -0.06522189 -0.19318713 -0.41779175 -0.20982976

In [8]:
import numpy as np

def sentence_to_embedding_word2vec(sentence, model):
    """
    Convertit une phrase en un vecteur en prenant la moyenne des embeddings des mots.
    """
    words = word_tokenize(sentence)
    embeddings = [model.wv[word] for word in words if word in model.wv]
    
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)  # Vecteur nul si aucun mot n'est connu

# Exemple pour une question
example_question = qa_pairs[0]["cleaned_question"]
embedding = sentence_to_embedding_word2vec(example_question, word2vec_model)
print("Embedding pour la question :", embedding)

Embedding pour la question : [-0.29703715 -0.10997645  0.6723885  -0.5187385  -1.1827625  -0.28674906
  0.13030064  0.483415   -0.3570408   2.3250408  -0.36229476 -0.98983383
 -0.39648485  0.35604915  0.1036403   0.33078563 -0.93228894 -0.15271376
  0.12611455  0.28620782  1.01119     0.21919106 -0.35367754  0.16813844
 -0.05643511  0.87095636 -1.1673893   1.4794537   0.3324403  -0.78654665
  0.6304491   0.3594596   0.4901556   0.467652   -0.6386835  -0.6058442
  0.24971531  0.54240584  1.1502842  -0.07795169 -0.37954774 -0.9303717
  0.7454948   0.09752699 -0.6513335   0.7276263   0.23814158 -0.73803884
  0.10072187  1.1574663  -0.35827184  0.14173386  0.4308665   0.08105826
 -0.09741207  0.31968665  0.20693462 -0.23730046 -0.498636    0.6846457
  0.13330568  0.13805254 -0.24278581  1.1326233   0.20357807  0.07394454
 -0.06791101  0.34911087  0.24798529  0.15796894 -0.9573367  -0.64676386
 -0.10805979 -0.4053034  -1.0223206   0.47645688  0.07105433  0.500444
  0.18836474 -0.09964224 -0

In [9]:
for pair in qa_pairs:
    pair["vector"] = sentence_to_embedding_word2vec(pair["cleaned_question"], word2vec_model)

print("Représentations vectorielles calculées avec Word2Vec.")

Représentations vectorielles calculées avec Word2Vec.


In [10]:
from scipy.spatial.distance import cosine

def cosine_similarity(vec1, vec2):
    """
    Calcule la similarité cosinus entre deux vecteurs.
    Retourne une valeur entre -1 (opposés) et 1 (identiques).
    """
    return 1 - cosine(vec1, vec2)

def find_most_similar_question(user_question, qa_pairs, model):
    """
    Trouve la question la plus similaire dans la base de données.
    """
    # Convertir la question de l'utilisateur en vecteur
    user_vector = sentence_to_embedding_word2vec(user_question, model)
    
    # Calculer les similarités cosinus
    similarities = [
        cosine_similarity(user_vector, pair["vector"]) for pair in qa_pairs
    ]
    
    # Identifier l'indice de la question la plus similaire
    best_match_index = similarities.index(max(similarities))
    
    return qa_pairs[best_match_index], max(similarities)

# Exemple d'utilisation
user_question = "How can I stay healthy?"
best_match, similarity_score = find_most_similar_question(user_question, qa_pairs, word2vec_model)
print(f"Question la plus similaire : {best_match['question']}")
print(f"Réponse correspondante : {best_match['answer']}")
print(f"Score de similarité : {similarity_score}")


Question la plus similaire : Give three tips for staying healthy.
Réponse correspondante : 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.

2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.

3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.
Score de similarité : 0.9296711478148632


c:\Users\Coco\Documents\Dossier Ecole\M1\Python\NLP\env\lib\site-packages\scipy\spatial\distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


In [2]:
def chatbot(qa_pairs, model):
    """
    Chatbot simple qui répond aux questions en utilisant la similarité cosinus.
    """
    print("Chatbot : Posez votre question ou tapez 'exit' pour quitter.")
    while True:
        user_input = st.text_input("Votre question, key="question")
        if user_input.lower() == "exit":
            print("Chatbot : À bientôt !")
            break
        
        # Trouver la meilleure correspondance
        best_match, similarity_score = find_most_similar_question(user_input, qa_pairs, model)
        
        # Donner une réponse
        st.write(f"Chatbot : {best_match['answer']} (similarité : {similarity_score:.2f})")

# Lancer le chatbot
chatbot(qa_pairs, word2vec_model)

Chatbot : Posez votre question ou tapez 'exit' pour quitter.
Chatbot : Here is a list of all the US Presidents since 1947:

1. Harry S. Truman (1945-1953)
2. Dwight D. Eisenhower (1953-1961)
3. John F. Kennedy (1961-1963)
4. Lyndon B. Johnson (1963-1969)
5. Richard Nixon (1969-1974)
6. Gerald Ford (1974-1977)
7. Jimmy Carter (1977-1981)
8. Ronald Reagan (1981-1989)
9. George H. W. Bush (1989-1993)
10. Bill Clinton (1993-2001)
11. George W. Bush (2001-2009)
12. Barack Obama (2009-2017)
13. Donald Trump (2017-2021)
14. Joe Biden (2021-current) (similarité : 0.75)
Chatbot : Here is a list of all the US Presidents since 1947:

1. Harry S. Truman (1945-1953)
2. Dwight D. Eisenhower (1953-1961)
3. John F. Kennedy (1961-1963)
4. Lyndon B. Johnson (1963-1969)
5. Richard Nixon (1969-1974)
6. Gerald Ford (1974-1977)
7. Jimmy Carter (1977-1981)
8. Ronald Reagan (1981-1989)
9. George H. W. Bush (1989-1993)
10. Bill Clinton (1993-2001)
11. George W. Bush (2001-2009)
12. Barack Obama (2009-2017)
13.